In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.jasper as jasper
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [4]:
config = malaya_speech.config.ctc_featurizer_config
featurizer = malaya_speech.tf_featurization.STTFeaturizer(**config)

In [5]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v, 0)
v, length_v

(<tf.Tensor 'ExpandDims_2:0' shape=(1, ?, 80) dtype=float32>,
 <tf.Tensor 'ExpandDims_1:0' shape=(1,) dtype=int32>)

In [6]:
model = jasper.Model(v, length_v)
model.logits


Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


{'outputs': <tf.Tensor 'w2l_encoder/dropout_52/mul_1:0' shape=(1, ?, 1024) dtype=float32>,
 'src_length': <tf.Tensor 'w2l_encoder/floordiv_104:0' shape=(1,) dtype=int32>}

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [8]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [9]:
l = sess.run(model.logits, feed_dict = {i: y})
l

{'outputs': array([[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.3219503 , 0.34961694, 1.5569376 , ..., 0.        ,
          0.        , 0.8924169 ],
         [0.        , 0.        , 1.0355753 , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 1.7037473 , ..., 0.6412966 ,
          0.        , 0.        ],
         [0.        , 0.        , 0.6107785 , ..., 0.        ,
          0.        , 0.        ],
         [0.40567687, 0.        , 0.6382924 , ..., 0.        ,
          1.1049418 , 0.12833233]]], dtype=float32),
 'src_length': array([281], dtype=int32)}

In [10]:
l = sess.run(model.logits, feed_dict = {i: y1})
l

{'outputs': array([[[1.7470075e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 1.6175895e-03, 1.1887778e-01],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          1.2271327e+00, 0.0000000e+00, 0.0000000e+00],
         [2.1354647e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [2.4050746e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 9.2267400e-01],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [8.0310202e-01, 7.1924508e-01, 8.1529897e-01, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00]]], dtype=float32),
 'src_length': array([175], dtype=int32)}